Anmerkung: Auf dem DKRZ Jupyterhub wähle oben rechts den Kernel ESMValTool

# Wo hat es wann gebrannt?

## Gib hier das Jahr an, das Dich interesseriert. 
Prinzipiell kannst ein Jahr zwischen 2000 und 2023 angeben.

## Nun laden wir die benötigten Apps.

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os
#-- auf dem DKRZ Supercomputer gibt es eine App noch nicht, diese muss erst installiert werden
try:
    host = os.uname()[1].split('.')[-2]
except (IndexError, ValueError):
    host = 'null'
if host == 'dkrz':                   #-- auf dem DKRZ Supercomputer gibt es 
    # Installiere App nbformat für die Gestaltung der Abbildung
    !pip install --user nbformat --quiet
    !pip install --user kaleido --quiet
    sys.path.append('../../.local/bin') #-- sage dem Computer, wo er die Apps findet

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
init_notebook_mode(connected=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Öffnen der Dateien mit der Feuerbeobachtungen

Die MODIS Feuerbeobachtungen kommen vom NASA Datenarchiv https://firms.modaps.eosdis.nasa.gov/country/.  
Dort können jährliche, zip-komprimierte Dateien heruntergeladen werden, die Daten für alle Länder enthalten. 
Der Einfachheit halber wurden die zip-Dateien vorab heruntergeladen, dekomprimiert, und so prozessiert, dass nur zwei csv-Dateien abgespeichert wurden. Letztere enthalten alle MODIS Feuerbeobachtungen für den Zeitraum 2001 bis 2023 für <br>
(a) Germany<br>
(b) Canada<br>
Die Dateinamen lauten<br>
modis_2000-2023_${LAND}.csv 

In [ ]:
country       = 'Canada'
datei      = 'modis_2001-2023_' + country + '.csv'
pfad       = '../modis_2001-2023'
feuerdaten = pd.read_csv(pfad + '/' + datei, usecols=['latitude','longitude','acq_date','acq_time','frp','type'])

## Hier passen wir die Daten etwas an und wählen nur Vegetationsbrände aus.
Bei jedem Feuer gibt es eine Typbeschreibung: Handelt es sich wahrscheinlich um ein Vegetationsbrand oder etwas anderes? Wir wählen nur Feuer aus, die vermutlich echte Vegetationsbrände sind.

In [ ]:
feuerdaten['Datum']    = pd.to_datetime(feuerdaten['acq_date'])
feuerdaten['Uhrzeit']  = pd.to_datetime(feuerdaten['acq_time'], format='%H%M').dt.strftime('%H')
feuerdaten.drop(['acq_time','acq_date'], axis='columns', inplace=True)

In [ ]:
feuerdaten['Monat_Jahr'] = feuerdaten['Datum'].dt.to_period('M').dt.strftime('%b-%Y').str.upper()
feuerdaten['Jahr'] = feuerdaten['Datum'].dt.strftime('%Y').str.upper()


In [ ]:
feuerdaten.rename({'latitude':'Breitengrad', 'longitude':'Längengrad', 'frp':'Feuerintensität', 'type':'Typ'}, axis='columns', inplace=True)
feuerdaten['Typ'].replace(to_replace=[0, 2], value=['Vegetationsbrand', 'Anderes'],inplace=True)
feuerdaten=feuerdaten[feuerdaten['Typ'] != 'Anderes']

### Wie sehen die Daten nun aus (erste 5 Zeilen)?

In [ ]:
feuerdaten.head(5)

###  Wie viele Vegetationsbrände gab es in dem Land zwischen 2001 bis 2023?

In [ ]:
print(feuerdaten['Typ'].value_counts().tolist()[0])

In [ ]:
feuerdaten['Feuerintensität'].max()

# Nun zeichnen wir eine Deutschlandkarte mit den Feuern. 

### Hier bestimmen wir die Farben des Farbbalkens.

In [ ]:
cmap = [[0, 'black'], [0.5, 'red'],[1, 'yellow']]

Hier könntet ihr auch andere Farben auf Englisch wählen. 
Wichtig ist, dass der englische Farbname klein geschrieben ist.
Ihr könntet auch eine Regenbogenfarbskala verwenden. Dann müsst ihr nur schreiben:<br>
cmap= 'rainbow'

### Hier wird die Abbildung gemacht:

In [ ]:
vmin, vmax = 0, 50
if country == 'Germany':
    land      = 'Deutschland'
    centerlat = 51.17  # Breitengrad von Zentrum Deutschlands
    centerlon = 10.45  # Längengrad von Zentrum Deutschlands
    vmin, vmax = 0, 50
    breite, hoehe = 700, 800
    zoom          = 5
elif country == 'Canada':
    land      = 'Kanada'
    centerlat = 56.13    # Breitengrad für das Zentrum Kanadas
    centerlon = -106.35  # Längengrad für das Zentrum Kanadas
    vmin, vmax = 0, 500
    breite, hoehe = 900, 500
    zoom          = 2.8

In [ ]:
animationszeitschritt = 'Jahr'   #-- alternativ:  'Monat_Jahr', 'Datum', 'Uhzeit'
feuerdaten.sort_values(by=animationszeitschritt, inplace=True)

In [ ]:
fig = px.scatter_mapbox(feuerdaten, 
                        lat = 'Breitengrad', 
                        lon = 'Längengrad', 
                        color = 'Feuerintensität', 
                        size = 'Feuerintensität',
                        color_continuous_scale  = cmap,
                        opacity = 0.5,
                        range_color = [vmin, vmax],
                        hover_data=['Typ'],
                        animation_frame= animationszeitschritt,  
                        center = {'lat': centerlat, 'lon': centerlon}, 
                        zoom = zoom,
                        mapbox_style = 'carto-positron')
fig.update_layout(
    title='MODIS Feuerdetektionen in ' + land, title_x=0.5,
    width=breite, height=hoehe,
    legend={'yanchor': 'top', 'xanchor':'left', 'y':1, 'x': 0.1},
    coloraxis={'colorbar':{'title':'Feuerintensität [MW]', 'ticks':'inside'}})
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 120

output_dir  = '../output'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f'MODIS_Feuer_{land}_{animationszeitschritt}.html')

pio.write_html(fig, file=output_file)

## Berechne zeitaggregiertes frp (MW)

In [ ]:
sum_timeries_MW = feuerdaten.groupby('Jahr')["Feuerintensität"].sum() 

## Schätze verbrannte Biomasse ab

In [ ]:
#conversion_factor_kg_MJ           = 0.356        # Wooster et al. (2025) Conversion factor of mass of combusted dry biomass and released fire radiative energy 0.356 kg MJ-1  experimental study with Miscanthus grass
conversion_factor_kg_MJ           = 1.55         # Kaiser et al. (2012)  Conversion factor of mass of combusted dry biomass and released fire radiative energy 0.1.55 kg MJ-1 for EFOS (extratropical forest with organic soil); empirical value by relating GFED dry matter burned estimates with GFAS FRP
fire_duration_estimated_sec       = 4 * 60 * 60  # own estimate average fire duration is 4 hours, conservative guess 

In [ ]:
sum_timeries_mio_tons   = sum_timeries_MW * fire_duration_estimated_sec * conversion_factor_kg_MJ * 1.e-9

In [ ]:
sum_timeries_mio_tons

In [ ]:

fontsize_normal = 14
fig, ax = plt.subplots(figsize=(12, 4))
sum_timeries_mio_tons.plot(kind="bar",  ax=ax, width=0.9, color ='blue').grid(axis='y')
ax.tick_params(labelsize=fontsize_normal)              
xmin, xmax = ax.get_xlim()
# pad bars on both sides a bit and draw the grid behind the bars
_ = ax.set(xlim=(xmin-0.25, xmax+0.25), axisbelow=True);
_ = plt.ylabel("Biomass burned (million tons)", fontsize=fontsize_normal + 2 , labelpad = 10);
_ = plt.xlabel("Year", fontsize=fontsize_normal + 2, labelpad = 10); 
_ = plt.title("Annual MODIS-derived biomass burned in " + country, fontsize = 22);

plt.savefig(f"MODIS_Annual_biomass_burned_{country}.png", bbox_inches='tight')

### How much CO2 (in million tons) was emitted by the fires in a selected year?

In [ ]:
myyear = "2023"

In [ ]:
co2_emission_factor =  1572       #-- g CO2 per kg biomass burned, Kaiser et al. 2012


In [ ]:
sum_timeries_mio_tons.loc[myyear] * co2_emission_factor * 1.e-3

....According to https://www.statista.com/statistics/209619/canadian-co2-emissions/#:~:text=Carbon%20dioxide%20emissions%20in%20Canada%201960%2D2022&text=Canada's%20carbon%20dioxide%20emissions%20from,tons%20(MtCO%E2%82%82)%20in%202022.
 Canada’s carbon dioxide emissions from fossil fuel combustion and industrial processes totaled 548 million metric tons (MtCO₂) in 2022